## Importieren der Module

In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pathlib import Path  
import glob

-----------

## Pfad einlesen

In [137]:
while True: 
    input_path = input("Was soll untersucht werden? allwords, nouns, adjectives oder verbs?").lower()
    
    if input_path == "allwords":
        directory_path = ["/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/all_words/clean_trump"]
        break
    elif input_path == "nouns":
        directory_path = ["/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_trump"]
        break
    elif input_path == "verbs":
        directory_path = ["/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/verbs/clean_trump"]
        break
    elif input_path == "adjectives":
        directory_path = ["/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/adjectives/clean_trump"]
        break
    else: 
        print("Falsche Eingabe!")

text_files = []
for path in directory_path:
    text_files.extend(glob.glob(f"{path}/*.txt"))

print(len(text_files))
text_files[:10]

Was soll untersucht werden? allwords, nouns, adjectives oder verbs? verbs


101


['/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/verbs/clean_trump/Donald Trump Rally in Schnecksville Pennsylvania 04_16_24.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/verbs/clean_trump/Donald Trump Speech Transcript: Kabul Bombing Would Not Have Happened If I Were Your President 08_26_21.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/verbs/clean_trump/Donald Trump Speech in Davenport, Iowa 3_13_23 Transcript 03_14_23.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/verbs/clean_trump/Trump Remarks after Dominant Super Tuesday Performance 03_06_24.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/verbs/clean_trump/Donald Trump White House Press Conference as Election Counts Continue Transcript 5 November 11_05_20.txt',
 "/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/verbs/clean_trump/FULL SPEECH: Trump Pledges To Enact The 'Largest Deportation In The History Of Our Country' In CPAC Sp

In [138]:
text_titles = [Path(text).stem for text in text_files]
print(len(text_titles))
text_titles[:10]

101


['Donald Trump Rally in Schnecksville Pennsylvania 04_16_24',
 'Donald Trump Speech Transcript: Kabul Bombing Would Not Have Happened If I Were Your President 08_26_21',
 'Donald Trump Speech in Davenport, Iowa 3_13_23 Transcript 03_14_23',
 'Trump Remarks after Dominant Super Tuesday Performance 03_06_24',
 'Donald Trump White House Press Conference as Election Counts Continue Transcript 5 November 11_05_20',
 "FULL SPEECH: Trump Pledges To Enact The 'Largest Deportation In The History Of Our Country' In CPAC Speech",
 'Donald Trump Delivers Keynote Speech at New Hampshire GOP Annual Meeting Transcript 01_30_23',
 'Donald Trump’s full NRA speech 5_27_22 Transcript 05_29_22',
 'Donald Trump Speaks at Rally in Ohio 03_18_24',
 'Former US President Donald Trump Delivers Speech Ahead of Independence Day Transcript 07_03_23']

--------------

## TF-IDF berechnen

In [139]:
tfidf_vectorizer = TfidfVectorizer(input='filename')

In [140]:
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)

In [141]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names_out())

In [142]:
if input_path in ("allwords", "nouns"):
    tfidf_slice = tfidf_df[['immigration', 'illegal']]
    print(tfidf_slice.sort_index().round(decimals=2))
else:
    print("Keine Substantive enthalten!")

Keine Substantive enthalten!


In [143]:
tfidf_df.stack().reset_index()

,level_0,level_1,0
0,Donald Trump Rally in Schnecksville Pennsylvan...,abandon,0.0
1,Donald Trump Rally in Schnecksville Pennsylvan...,abbreviate,0.0
2,Donald Trump Rally in Schnecksville Pennsylvan...,abide,0.0
3,Donald Trump Rally in Schnecksville Pennsylvan...,abolish,0.0
4,Donald Trump Rally in Schnecksville Pennsylvan...,absorb,0.0
...,...,...,...
180684,Iowa Town Hall with Donald Trump 1_10_24 Trans...,wrongdoe,0.0
180685,Iowa Town Hall with Donald Trump 1_10_24 Trans...,wyome,0.0
180686,Iowa Town Hall with Donald Trump 1_10_24 Trans...,yearn,0.0
180687,Iowa Town Hall with Donald Trump 1_10_24 Trans...,zeldin,0.0


In [144]:
tfidf_df = tfidf_df.stack().reset_index()

In [145]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

In [146]:
tfidf_df.columns

Index(['document', 'term', 'tfidf'], dtype='object')

In [147]:
tfidf_df.sort_values(by=['tfidf','document'], ascending=[False,False]).groupby(['tfidf']).head(10)

,document,term,tfidf
91054,Donald Trump Election Day Interview Transcript...,think,0.842502
146513,Donald Trump Remarks to RNC Staff Transcript A...,think,0.814194
26650,Tucker on X Debate Night Interview with Donald...,think,0.729365
180504,Iowa Town Hall with Donald Trump 1_10_24 Trans...,think,0.711637
166192,"Donald Trump Rally In Greenwood, Nebraska 5_1_...",think,0.684857
...,...,...,...
25051,Tucker on X Debate Night Interview with Donald...,abuse,0.000000
25052,Tucker on X Debate Night Interview with Donald...,accelerate,0.000000
25053,Tucker on X Debate Night Interview with Donald...,accept,0.000000
25054,Tucker on X Debate Night Interview with Donald...,accompany,0.000000


In [148]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

In [149]:
top_tfidf[top_tfidf['term'].str.contains('war')]

,document,term,tfidf


---------------

## TF-IDF visualisieren

In [150]:
import altair as alt
import numpy as np

#terme die gesondert mit rotem punkt ausgezeichnet werden
term_list = ['war', 'peace']

#randomness fürs ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

#visualisierungen mit ranking
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

#heatmap definieren
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

#roter punkt für oben festgelegte terme definieren
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

#text labels mit weiß für dunklere heatmap-einfärbungen
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

#visualisierungen darstellen
(heatmap + circle + text).properties(width = 600)

alt.LayerChart(...)